# Foundation PLR: Reproducibility Tutorial

**Authors**: [Your Name]
**Last Updated**: 2026-01-19

---

## Overview

This tutorial demonstrates how to reproduce the classification and statistical analyses from the Foundation PLR study without access to the raw clinical data.

### What You Can Reproduce

| Analysis | Reproducible | Data Required |
|----------|--------------|---------------|
| Classification training | ✅ Yes | `foundation_plr_features.db` |
| Statistical analyses | ✅ Yes | `foundation_plr_results.db` |
| Calibration curves | ✅ Yes | `foundation_plr_results.db` |
| Decision curve analysis | ✅ Yes | `foundation_plr_results.db` |
| Uncertainty propagation | ✅ Yes | `foundation_plr_features.db` |
| Publication figures | ✅ Yes | Either database |

### What Cannot Be Reproduced

Due to clinical data privacy restrictions:
- Raw PLR signal preprocessing
- Anomaly detection model training
- Imputation model training
- Feature extraction from raw signals

### Prerequisites

```bash
pip install duckdb pandas numpy scipy scikit-learn matplotlib seaborn
pip install xgboost catboost statsmodels pingouin
```

---

## 1. Setup and Imports

In [ ]:
# Standard library
import sys
import warnings
from pathlib import Path

# Data handling
import duckdb
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss,
    roc_curve, precision_recall_curve, confusion_matrix,
    classification_report
)
from sklearn.calibration import calibration_curve

# Optional: Advanced classifiers
try:
    from xgboost import XGBClassifier
    HAS_XGBOOST = True
except ImportError:
    HAS_XGBOOST = False
    print("XGBoost not available. Install with: pip install xgboost")

try:
    from catboost import CatBoostClassifier
    HAS_CATBOOST = True
except ImportError:
    HAS_CATBOOST = False
    print("CatBoost not available. Install with: pip install catboost")

# Project imports (add project root to path)
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Foundation PLR statistics modules
from src.stats import (
    # Effect sizes
    cohens_d, hedges_g, partial_eta_squared,
    # Multiple comparison correction
    benjamini_hochberg, bonferroni,
    # Bootstrap
    bca_bootstrap_ci, percentile_bootstrap_ci,
    # Calibration
    calibration_slope_intercept, brier_decomposition,
    # Clinical utility
    net_benefit, decision_curve_analysis,
    # Uncertainty propagation
    monte_carlo_classifier_uncertainty,
    clinical_decision_stability,
    sensitivity_analysis_delta,
)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=FutureWarning)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

print("Setup complete!")
print(f"Project root: {PROJECT_ROOT}")

---

## 2. Loading Data from DuckDB

The shared databases contain:
- **`foundation_plr_features.db`**: Hand-crafted PLR features (de-identified)
- **`foundation_plr_results.db`**: All classifier predictions and metrics

In [ ]:
# Define paths to shared databases
DATA_DIR = PROJECT_ROOT / "data"
FEATURES_DB = DATA_DIR / "foundation_plr_features.db"
RESULTS_DB = DATA_DIR / "foundation_plr_results.db"

# Check which databases are available
print("Checking data availability:")
print(f"  Features DB: {'✅ Found' if FEATURES_DB.exists() else '❌ Not found'}")
print(f"  Results DB:  {'✅ Found' if RESULTS_DB.exists() else '❌ Not found'}")

if not FEATURES_DB.exists() and not RESULTS_DB.exists():
    print("\n⚠️  Neither database found. Download from [repository URL] or run export script.")
    print("   This tutorial will use synthetic demo data instead.")
    USE_DEMO_DATA = True
else:
    USE_DEMO_DATA = False

### 2.1 Exploring the Database Schema

In [ ]:
def explore_duckdb(db_path: Path) -> dict:
    """
    Explore a DuckDB database and return schema information.
    
    Parameters
    ----------
    db_path : Path
        Path to DuckDB database file
        
    Returns
    -------
    dict
        Database schema information
    """
    if not db_path.exists():
        return {"error": "Database not found"}
    
    with duckdb.connect(str(db_path), read_only=True) as con:
        # Get all tables
        tables = con.execute("SHOW TABLES").df()
        
        schema = {}
        for table_name in tables['name']:
            # Get column info
            columns = con.execute(f"DESCRIBE {table_name}").df()
            
            # Get row count
            count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
            
            schema[table_name] = {
                'columns': columns['column_name'].tolist(),
                'types': columns['column_type'].tolist(),
                'row_count': count
            }
    
    return schema

# Explore available databases
if FEATURES_DB.exists():
    print("\n=== Features Database Schema ===")
    features_schema = explore_duckdb(FEATURES_DB)
    for table, info in features_schema.items():
        print(f"\nTable: {table} ({info['row_count']:,} rows)")
        print(f"  Columns: {', '.join(info['columns'][:5])}..." if len(info['columns']) > 5 else f"  Columns: {', '.join(info['columns'])}")

if RESULTS_DB.exists():
    print("\n=== Results Database Schema ===")
    results_schema = explore_duckdb(RESULTS_DB)
    for table, info in results_schema.items():
        print(f"\nTable: {table} ({info['row_count']:,} rows)")
        print(f"  Columns: {', '.join(info['columns'][:5])}..." if len(info['columns']) > 5 else f"  Columns: {', '.join(info['columns'])}")

### 2.2 Loading Features for Classification

In [ ]:
def load_features(
    db_path: Path,
    source_name: str = None,
    split: str = None,
) -> tuple:
    """
    Load features and labels from DuckDB.
    
    Parameters
    ----------
    db_path : Path
        Path to features database
    source_name : str, optional
        Filter by pipeline configuration
    split : str, optional
        Filter by data split ('train', 'val', 'test')
        
    Returns
    -------
    X : np.ndarray
        Feature matrix (n_samples, n_features)
    y : np.ndarray
        Labels (n_samples,)
    feature_names : list
        Feature column names
    metadata : pd.DataFrame
        Subject metadata (subject_id, eye, split)
    """
    with duckdb.connect(str(db_path), read_only=True) as con:
        # Build query with optional filters
        query = """
            SELECT f.*, m.has_glaucoma, m.split, m.subject_id, m.eye
            FROM plr_features f
            JOIN feature_metadata m
                ON f.subject_id = m.subject_id AND f.eye = m.eye
            WHERE 1=1
        """
        
        if source_name:
            query += f" AND f.source_name = '{source_name}'"
        if split:
            query += f" AND m.split = '{split}'"
            
        df = con.execute(query).df()
    
    # Separate metadata from features
    metadata_cols = ['subject_id', 'eye', 'source_name', 'has_glaucoma', 'split']
    feature_cols = [c for c in df.columns if c not in metadata_cols]
    
    X = df[feature_cols].values.astype(np.float32)
    y = df['has_glaucoma'].values.astype(int)
    metadata = df[['subject_id', 'eye', 'split']]
    
    return X, y, feature_cols, metadata


def create_demo_data(n_samples: int = 200, n_features: int = 15, random_state: int = 42):
    """
    Create synthetic demo data for tutorial demonstration.
    
    The synthetic features mimic PLR characteristics:
    - Baseline diameter (normally ~3-6 mm)
    - Constriction amplitude (typically 0.5-2.0 mm)
    - Latency features (typically 200-500 ms)
    - Velocity features (mm/s)
    - PIPR features (percentage)
    """
    rng = np.random.default_rng(random_state)
    
    # Class labels with ~30% prevalence (like glaucoma in clinical settings)
    y = rng.binomial(1, 0.3, n_samples)
    
    # Feature names mimicking PLR features
    feature_names = [
        'baseline_diameter', 'constriction_amplitude', 'constriction_amplitude_rel',
        'max_constriction_diameter', 'latency_to_constriction', 'latency_75pct',
        'time_to_redilation', 'max_constriction_velocity', 'mean_constriction_velocity',
        'max_redilation_velocity', 'pipr_6s', 'pipr_10s', 'recovery_time',
        'constriction_duration', 'auc_constriction'
    ][:n_features]
    
    # Generate correlated features with class-dependent means
    X = np.zeros((n_samples, n_features))
    
    # Glaucoma typically shows: smaller PIPR, slower velocities, reduced amplitudes
    for i in range(n_samples):
        if y[i] == 1:  # Glaucoma
            base_shift = -0.3
        else:  # Control
            base_shift = 0.3
        
        X[i, :] = rng.normal(base_shift, 1.0, n_features)
        
        # Add realistic feature correlations
        X[i, 0] = rng.normal(4.5 + base_shift * 0.5, 0.8)  # baseline_diameter ~3-6mm
        X[i, 1] = rng.normal(1.2 + base_shift * 0.3, 0.4)  # constriction_amplitude
        X[i, 4] = rng.normal(350 - base_shift * 30, 50)     # latency ~250-450ms
    
    # Create metadata
    metadata = pd.DataFrame({
        'subject_id': [f'S{i:04d}' for i in range(n_samples)],
        'eye': rng.choice(['OD', 'OS'], n_samples),
        'split': np.where(
            np.arange(n_samples) < int(0.7 * n_samples),
            'train',
            np.where(np.arange(n_samples) < int(0.85 * n_samples), 'val', 'test')
        )
    })
    
    return X.astype(np.float32), y, feature_names, metadata


# Load data (real or demo)
if USE_DEMO_DATA or not FEATURES_DB.exists():
    print("Using synthetic demo data...")
    X, y, feature_names, metadata = create_demo_data(n_samples=300)
else:
    print(f"Loading features from {FEATURES_DB}...")
    X, y, feature_names, metadata = load_features(FEATURES_DB)

print(f"\nData loaded:")
print(f"  Samples: {X.shape[0]}")
print(f"  Features: {X.shape[1]}")
print(f"  Class distribution: {np.bincount(y)} (Control: {np.sum(y==0)}, Glaucoma: {np.sum(y==1)})")
print(f"  Features: {feature_names[:5]}...")

---

## 3. Re-running Classification

This section demonstrates training classifiers on the shared features and comparing results with the published values.

### 3.1 Define Classifiers

In [ ]:
def get_classifiers() -> dict:
    """
    Get dictionary of classifiers used in the study.
    
    Returns
    -------
    dict
        Mapping from classifier name to (model, hyperparameters)
    """
    classifiers = {
        'LogisticRegression': LogisticRegression(
            max_iter=1000,
            solver='lbfgs',
            random_state=42,
            class_weight='balanced'
        ),
    }
    
    if HAS_XGBOOST:
        classifiers['XGBoost'] = XGBClassifier(
            n_estimators=100,
            max_depth=5,
            learning_rate=0.1,
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=42,
            n_jobs=-1
        )
    
    if HAS_CATBOOST:
        classifiers['CatBoost'] = CatBoostClassifier(
            iterations=100,
            depth=5,
            learning_rate=0.1,
            random_state=42,
            verbose=False
        )
    
    return classifiers

classifiers = get_classifiers()
print(f"Available classifiers: {list(classifiers.keys())}")

### 3.2 Cross-Validated Evaluation

In [ ]:
def evaluate_classifier(
    clf,
    X: np.ndarray,
    y: np.ndarray,
    n_folds: int = 5,
    n_bootstrap: int = 1000,
    random_state: int = 42
) -> dict:
    """
    Comprehensive classifier evaluation with bootstrap confidence intervals.
    
    Parameters
    ----------
    clf : sklearn estimator
        Classifier with fit/predict_proba methods
    X : np.ndarray
        Feature matrix
    y : np.ndarray
        Labels
    n_folds : int
        Number of CV folds
    n_bootstrap : int
        Bootstrap iterations for CIs
    random_state : int
        Random seed
        
    Returns
    -------
    dict
        Evaluation results with metrics and CIs
    """
    from sklearn.base import clone
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    
    # Storage for predictions
    y_prob_all = np.zeros(len(y))
    y_pred_all = np.zeros(len(y), dtype=int)
    fold_metrics = []
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Clone and fit
        model = clone(clf)
        model.fit(X_train, y_train)
        
        # Predict
        y_prob = model.predict_proba(X_test)[:, 1]
        y_pred = (y_prob >= 0.5).astype(int)
        
        y_prob_all[test_idx] = y_prob
        y_pred_all[test_idx] = y_pred
        
        # Fold metrics
        fold_metrics.append({
            'fold': fold,
            'auroc': roc_auc_score(y_test, y_prob),
            'auprc': average_precision_score(y_test, y_prob),
            'brier': brier_score_loss(y_test, y_prob),
        })
    
    # Overall metrics
    auroc = roc_auc_score(y, y_prob_all)
    auprc = average_precision_score(y, y_prob_all)
    brier = brier_score_loss(y, y_prob_all)
    
    # Bootstrap CIs
    rng = np.random.default_rng(random_state)
    boot_aurocs = []
    boot_auprcs = []
    
    for _ in range(n_bootstrap):
        idx = rng.choice(len(y), len(y), replace=True)
        if len(np.unique(y[idx])) < 2:
            continue
        boot_aurocs.append(roc_auc_score(y[idx], y_prob_all[idx]))
        boot_auprcs.append(average_precision_score(y[idx], y_prob_all[idx]))
    
    return {
        'auroc': auroc,
        'auroc_ci': (np.percentile(boot_aurocs, 2.5), np.percentile(boot_aurocs, 97.5)),
        'auprc': auprc,
        'auprc_ci': (np.percentile(boot_auprcs, 2.5), np.percentile(boot_auprcs, 97.5)),
        'brier': brier,
        'y_prob': y_prob_all,
        'y_pred': y_pred_all,
        'fold_metrics': pd.DataFrame(fold_metrics),
    }


# Evaluate all classifiers
results = {}
print("Evaluating classifiers (5-fold CV with bootstrap CIs)...\n")

for name, clf in classifiers.items():
    print(f"  {name}...", end=' ')
    results[name] = evaluate_classifier(clf, X, y, n_bootstrap=500)
    auroc = results[name]['auroc']
    ci = results[name]['auroc_ci']
    print(f"AUROC = {auroc:.4f} (95% CI: [{ci[0]:.4f}, {ci[1]:.4f}])")

print("\nEvaluation complete!")

### 3.3 Results Summary Table

In [ ]:
# Create results summary table
summary_data = []
for name, res in results.items():
    summary_data.append({
        'Classifier': name,
        'AUROC': f"{res['auroc']:.4f}",
        '95% CI': f"[{res['auroc_ci'][0]:.4f}, {res['auroc_ci'][1]:.4f}]",
        'AUPRC': f"{res['auprc']:.4f}",
        'Brier Score': f"{res['brier']:.4f}",
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*70)
print("CLASSIFICATION RESULTS SUMMARY")
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)

---

## 4. Statistical Analyses

This section demonstrates the biostatistical analyses used in the publication.

### 4.1 Calibration Analysis (STRATOS-Compliant)

In [ ]:
def analyze_calibration(y_true: np.ndarray, y_prob: np.ndarray, name: str) -> dict:
    """
    Comprehensive calibration analysis following STRATOS guidelines.
    
    Reports:
    - Calibration slope (target = 1.0)
    - Calibration intercept (calibration-in-the-large)
    - E:O ratio (Expected:Observed)
    - Brier score decomposition
    """
    # STRATOS-compliant calibration metrics
    cal_result = calibration_slope_intercept(y_true, y_prob)
    brier_result = brier_decomposition(y_true, y_prob)
    
    print(f"\nCalibration Analysis: {name}")
    print("-" * 40)
    print(f"Calibration slope:     {cal_result.slope:.4f}")
    print(f"  Interpretation:      {'Well-calibrated' if 0.8 < cal_result.slope < 1.2 else 'Miscalibrated'}")
    print(f"  (slope < 1 = overfitting, slope > 1 = underfitting)")
    print(f"")
    print(f"Calibration intercept: {cal_result.intercept:.4f}")
    print(f"E:O ratio:             {cal_result.e_o_ratio:.4f}")
    print(f"  (E:O > 1 = over-prediction, E:O < 1 = under-prediction)")
    print(f"")
    print(f"Brier score:           {cal_result.brier_score:.4f}")
    print(f"  - Reliability:       {brier_result.scalars.get('reliability', np.nan):.4f}")
    print(f"  - Resolution:        {brier_result.scalars.get('resolution', np.nan):.4f}")
    print(f"  - Uncertainty:       {brier_result.scalars.get('uncertainty', np.nan):.4f}")
    
    return {
        'slope': cal_result.slope,
        'intercept': cal_result.intercept,
        'e_o_ratio': cal_result.e_o_ratio,
        'brier': cal_result.brier_score,
    }


# Analyze calibration for each classifier
calibration_results = {}
for name, res in results.items():
    calibration_results[name] = analyze_calibration(y, res['y_prob'], name)

### 4.2 Calibration Plot

In [ ]:
def plot_calibration_curves(results: dict, y_true: np.ndarray, n_bins: int = 10):
    """
    Plot calibration curves for all classifiers.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Left: Calibration curves
    ax1 = axes[0]
    ax1.plot([0, 1], [0, 1], 'k--', label='Perfectly calibrated', alpha=0.7)
    
    colors = plt.cm.Set2(np.linspace(0, 1, len(results)))
    for (name, res), color in zip(results.items(), colors):
        prob_true, prob_pred = calibration_curve(y_true, res['y_prob'], n_bins=n_bins, strategy='uniform')
        ax1.plot(prob_pred, prob_true, 's-', color=color, label=name, markersize=6)
    
    ax1.set_xlabel('Mean predicted probability')
    ax1.set_ylabel('Observed proportion')
    ax1.set_title('Calibration Curves')
    ax1.legend(loc='lower right')
    ax1.set_xlim([0, 1])
    ax1.set_ylim([0, 1])
    ax1.set_aspect('equal')
    
    # Right: Prediction distribution
    ax2 = axes[1]
    for (name, res), color in zip(results.items(), colors):
        ax2.hist(res['y_prob'][y_true == 0], bins=20, alpha=0.5, color=color, 
                 label=f'{name} (Control)', density=True)
        ax2.hist(res['y_prob'][y_true == 1], bins=20, alpha=0.5, color=color,
                 density=True, hatch='//', edgecolor='black', linewidth=0.5)
    
    ax2.set_xlabel('Predicted probability')
    ax2.set_ylabel('Density')
    ax2.set_title('Prediction Distributions by True Class')
    ax2.legend(['Control', 'Glaucoma'], loc='upper center')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'notebooks' / 'calibration_curves.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Saved: notebooks/calibration_curves.png")

plot_calibration_curves(results, y)

### 4.3 Decision Curve Analysis (Clinical Utility)

In [ ]:
def plot_decision_curves(results: dict, y_true: np.ndarray):
    """
    Decision Curve Analysis for clinical utility assessment.
    
    For glaucoma screening:
    - Threshold range: 1% to 30%
    - Lower thresholds favor sensitivity (screening)
    - Higher thresholds favor specificity (confirmation)
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Threshold range relevant for glaucoma screening
    threshold_range = (0.01, 0.30)
    
    # Baseline strategies
    thresholds = np.linspace(threshold_range[0], threshold_range[1], 30)
    prevalence = np.mean(y_true)
    
    # Treat-all net benefit
    nb_all = [prevalence - (1 - prevalence) * (pt / (1 - pt)) for pt in thresholds]
    nb_all = np.maximum(nb_all, 0)  # Cap at 0
    
    ax.plot(thresholds * 100, nb_all, 'k--', label='Treat All', linewidth=2)
    ax.axhline(y=0, color='gray', linestyle=':', label='Treat None', linewidth=2)
    
    # Model net benefits
    colors = plt.cm.Set2(np.linspace(0, 1, len(results)))
    for (name, res), color in zip(results.items(), colors):
        dca_df = decision_curve_analysis(
            y_true, res['y_prob'],
            threshold_range=threshold_range
        )
        ax.plot(dca_df['threshold'] * 100, dca_df['nb_model'], 
                color=color, label=name, linewidth=2)
    
    ax.set_xlabel('Threshold Probability (%)', fontsize=12)
    ax.set_ylabel('Net Benefit', fontsize=12)
    ax.set_title('Decision Curve Analysis for Glaucoma Screening', fontsize=14)
    ax.legend(loc='upper right')
    ax.set_xlim([threshold_range[0] * 100, threshold_range[1] * 100])
    ax.set_ylim([-0.05, max(0.15, prevalence + 0.05)])
    
    # Add clinical interpretation
    ax.axvspan(1, 10, alpha=0.1, color='green', label='_Screening zone')
    ax.axvspan(10, 30, alpha=0.1, color='blue', label='_Diagnosis zone')
    ax.text(5, ax.get_ylim()[1] * 0.9, 'Screening', ha='center', fontsize=9, color='darkgreen')
    ax.text(20, ax.get_ylim()[1] * 0.9, 'Diagnosis', ha='center', fontsize=9, color='darkblue')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'notebooks' / 'decision_curve_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Saved: notebooks/decision_curve_analysis.png")
    
    # Print interpretation
    print("\nInterpretation:")
    print("  - A model provides clinical utility when its curve is above both baselines")
    print("  - Net benefit = (TP/n) - (FP/n) × (threshold / (1 - threshold))")
    print(f"  - Prevalence in this data: {prevalence:.1%}")

plot_decision_curves(results, y)

### 4.4 Effect Sizes with Confidence Intervals

In [ ]:
def compute_classifier_comparison_effect_sizes(results: dict):
    """
    Compute effect sizes (Cohen's d) for pairwise classifier comparisons.
    
    Uses fold-level AUROC values for comparison.
    """
    classifiers = list(results.keys())
    comparisons = []
    
    for i, clf1 in enumerate(classifiers):
        for clf2 in classifiers[i+1:]:
            # Get fold-level AUROCs
            aurocs1 = results[clf1]['fold_metrics']['auroc'].values
            aurocs2 = results[clf2]['fold_metrics']['auroc'].values
            
            # Compute Cohen's d
            effect = cohens_d(aurocs1, aurocs2)
            
            comparisons.append({
                'Comparison': f"{clf1} vs {clf2}",
                'Mean Diff': f"{np.mean(aurocs1) - np.mean(aurocs2):.4f}",
                "Cohen's d": f"{effect.effect_size:.3f}",
                '95% CI': f"[{effect.ci_lower:.3f}, {effect.ci_upper:.3f}]",
                'Interpretation': effect.interpretation,
            })
    
    df = pd.DataFrame(comparisons)
    print("\n" + "="*70)
    print("CLASSIFIER COMPARISON: EFFECT SIZES")
    print("="*70)
    print(df.to_string(index=False))
    print("="*70)
    print("\nInterpretation guide:")
    print("  |d| < 0.2: negligible")
    print("  0.2 ≤ |d| < 0.5: small")
    print("  0.5 ≤ |d| < 0.8: medium")
    print("  |d| ≥ 0.8: large")
    
    return df

if len(results) > 1:
    effect_sizes_df = compute_classifier_comparison_effect_sizes(results)
else:
    print("Need multiple classifiers for effect size comparison.")

---

## 5. Uncertainty Propagation Analysis

This section demonstrates how feature uncertainty affects clinical decisions.

### 5.1 Monte Carlo Uncertainty Propagation

In [ ]:
def run_uncertainty_analysis(
    X: np.ndarray,
    clf,
    uncertainty_level: float = 0.1,
    n_simulations: int = 500
):
    """
    Run Monte Carlo uncertainty propagation analysis.
    
    Parameters
    ----------
    X : np.ndarray
        Feature matrix
    clf : trained classifier
        Must have predict_proba method
    uncertainty_level : float
        Feature uncertainty as fraction of feature std
    n_simulations : int
        Number of MC simulations
    """
    # Estimate feature uncertainties (fraction of feature standard deviation)
    feature_stds = np.std(X, axis=0)
    uncertainties = feature_stds * uncertainty_level
    
    print(f"\nRunning uncertainty analysis...")
    print(f"  Feature uncertainty: {uncertainty_level:.0%} of feature std")
    print(f"  MC simulations: {n_simulations}")
    
    # Define prediction function
    def predict_fn(X_perturbed):
        return clf.predict_proba(X_perturbed)[:, 1]
    
    # Run MC simulation
    mc_result = monte_carlo_classifier_uncertainty(
        X, uncertainties, predict_fn,
        n_simulations=n_simulations,
        random_state=42
    )
    
    # Analyze decision stability
    stability_result = clinical_decision_stability(
        mc_result.arrays['all_predictions'],
        threshold=0.5,
        stability_criterion='majority'
    )
    
    print(f"\nResults:")
    print(f"  Mean prediction uncertainty (std): {mc_result.scalars['mean_prediction_std']:.4f}")
    print(f"  Max prediction uncertainty (std): {mc_result.scalars['max_prediction_std']:.4f}")
    print(f"  Decision stability: {stability_result.decision_stability_pct:.1f}%")
    print(f"  Subjects with unstable decisions: {stability_result.n_unstable}")
    
    return mc_result, stability_result


# Train a classifier for uncertainty analysis
best_clf_name = max(results.keys(), key=lambda k: results[k]['auroc'])
print(f"Using {best_clf_name} for uncertainty analysis...")

# Fit on full data for this analysis
from sklearn.base import clone
trained_clf = clone(classifiers[best_clf_name])
trained_clf.fit(X, y)

# Run uncertainty analysis at different uncertainty levels
uncertainty_levels = [0.05, 0.10, 0.20]
uncertainty_results = {}

for level in uncertainty_levels:
    print(f"\n{'='*50}")
    print(f"UNCERTAINTY LEVEL: {level:.0%}")
    print(f"{'='*50}")
    mc_result, stability_result = run_uncertainty_analysis(X, trained_clf, level)
    uncertainty_results[level] = (mc_result, stability_result)

### 5.2 Sensitivity Analysis

In [ ]:
def run_sensitivity_analysis(X: np.ndarray, clf, feature_names: list):
    """
    Identify which features most affect predictions under uncertainty.
    """
    # Feature uncertainties
    uncertainties = np.std(X, axis=0) * 0.1
    
    def predict_fn(X_perturbed):
        return clf.predict_proba(X_perturbed)[:, 1]
    
    # Run sensitivity analysis
    sensitivity = sensitivity_analysis_delta(
        X, uncertainties, predict_fn,
        feature_names=feature_names
    )
    
    # Create visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Sort by sensitivity
    sorted_idx = np.argsort(sensitivity.sensitivity_normalized)[::-1]
    sorted_names = [feature_names[i] for i in sorted_idx]
    sorted_sens = sensitivity.sensitivity_normalized[sorted_idx]
    
    colors = plt.cm.RdYlGn_r(sorted_sens / max(sorted_sens))
    bars = ax.barh(range(len(sorted_names)), sorted_sens, color=colors)
    ax.set_yticks(range(len(sorted_names)))
    ax.set_yticklabels(sorted_names)
    ax.invert_yaxis()
    ax.set_xlabel('Normalized Sensitivity Index')
    ax.set_title('Feature Sensitivity Analysis\n(Which features most affect predictions under uncertainty)')
    
    # Add values on bars
    for i, (bar, val) in enumerate(zip(bars, sorted_sens)):
        ax.text(val + 0.01, bar.get_y() + bar.get_height()/2, 
                f'{val:.3f}', va='center', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'notebooks' / 'sensitivity_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Saved: notebooks/sensitivity_analysis.png")
    
    print(f"\nMost influential feature: {sensitivity.most_influential}")
    print(f"Top 3 features account for {sum(sorted_sens[:3]):.1%} of total sensitivity")
    
    return sensitivity

sensitivity = run_sensitivity_analysis(X, trained_clf, feature_names)

---

## 6. Extending the Analysis

This section shows how to add custom metrics, classifiers, or statistical tests.

### 6.1 Adding a Custom Metric

In [ ]:
def youden_j_index(y_true: np.ndarray, y_prob: np.ndarray) -> dict:
    """
    Compute Youden's J statistic and optimal threshold.
    
    J = Sensitivity + Specificity - 1
    
    The optimal threshold maximizes J.
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    
    # Youden's J
    j_scores = tpr - fpr
    optimal_idx = np.argmax(j_scores)
    
    return {
        'youden_j': j_scores[optimal_idx],
        'optimal_threshold': thresholds[optimal_idx],
        'sensitivity_at_optimal': tpr[optimal_idx],
        'specificity_at_optimal': 1 - fpr[optimal_idx],
    }

# Apply custom metric to all classifiers
print("\nYouden's J Index Analysis:")
print("-" * 60)
for name, res in results.items():
    j_result = youden_j_index(y, res['y_prob'])
    print(f"\n{name}:")
    print(f"  Youden's J: {j_result['youden_j']:.4f}")
    print(f"  Optimal threshold: {j_result['optimal_threshold']:.4f}")
    print(f"  At optimal: Sens={j_result['sensitivity_at_optimal']:.3f}, Spec={j_result['specificity_at_optimal']:.3f}")

### 6.2 Adding a Custom Statistical Test

In [ ]:
def delong_test(y_true: np.ndarray, y_prob1: np.ndarray, y_prob2: np.ndarray):
    """
    DeLong test for comparing two ROC curves.
    
    Tests H0: AUROC1 = AUROC2
    
    Note: This is a simplified implementation. For publication,
    consider using the 'pROC' R package via rpy2.
    """
    from scipy import stats
    
    auroc1 = roc_auc_score(y_true, y_prob1)
    auroc2 = roc_auc_score(y_true, y_prob2)
    
    # Bootstrap for variance estimate
    rng = np.random.default_rng(42)
    n_boot = 1000
    diffs = []
    
    for _ in range(n_boot):
        idx = rng.choice(len(y_true), len(y_true), replace=True)
        if len(np.unique(y_true[idx])) < 2:
            continue
        a1 = roc_auc_score(y_true[idx], y_prob1[idx])
        a2 = roc_auc_score(y_true[idx], y_prob2[idx])
        diffs.append(a1 - a2)
    
    # Test statistic
    se = np.std(diffs)
    z = (auroc1 - auroc2) / se if se > 0 else 0
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    
    return {
        'auroc1': auroc1,
        'auroc2': auroc2,
        'difference': auroc1 - auroc2,
        'z_statistic': z,
        'p_value': p_value,
        'significant': p_value < 0.05
    }

# Compare classifiers if we have multiple
if len(results) >= 2:
    clf_names = list(results.keys())
    print("\nDeLong Test (Comparing ROC Curves):")
    print("-" * 60)
    
    for i, name1 in enumerate(clf_names):
        for name2 in clf_names[i+1:]:
            test = delong_test(y, results[name1]['y_prob'], results[name2]['y_prob'])
            sig = "*" if test['significant'] else ""
            print(f"\n{name1} vs {name2}:")
            print(f"  AUROC difference: {test['difference']:.4f}")
            print(f"  p-value: {test['p_value']:.4f}{sig}")

### 6.3 Exporting Results for Publication

In [ ]:
def export_results_to_latex(results: dict, calibration_results: dict, output_path: Path):
    """
    Export results as LaTeX tables for publication.
    """
    # Main results table
    latex_content = r"""
\begin{table}[htbp]
\centering
\caption{Classification Performance Comparison}
\label{tab:classification-results}
\begin{tabular}{lcccc}
\toprule
\textbf{Classifier} & \textbf{AUROC} & \textbf{95\% CI} & \textbf{AUPRC} & \textbf{Brier} \\
\midrule
"""
    
    for name, res in results.items():
        auroc = res['auroc']
        ci = res['auroc_ci']
        auprc = res['auprc']
        brier = res['brier']
        latex_content += f"{name} & {auroc:.3f} & [{ci[0]:.3f}, {ci[1]:.3f}] & {auprc:.3f} & {brier:.3f} \\\\\n"
    
    latex_content += r"""
\bottomrule
\end{tabular}
\end{table}
"""
    
    with open(output_path, 'w') as f:
        f.write(latex_content)
    
    print(f"LaTeX table saved to: {output_path}")
    return latex_content

# Export
latex_output = export_results_to_latex(
    results, 
    calibration_results,
    PROJECT_ROOT / 'notebooks' / 'classification_results.tex'
)
print("\nGenerated LaTeX:")
print(latex_output)

---

## 7. Summary and Next Steps

This tutorial demonstrated:

1. **Loading shared data** from DuckDB databases
2. **Re-running classification** with multiple algorithms
3. **STRATOS-compliant calibration analysis**
4. **Decision curve analysis** for clinical utility
5. **Uncertainty propagation** via Monte Carlo simulation
6. **Effect size computation** with confidence intervals
7. **Extending the analysis** with custom metrics and tests

### Where to Modify Code

| Task | Location |
|------|----------|
| Add new classifier | `get_classifiers()` function |
| Add new metric | `evaluate_classifier()` function |
| Change statistical tests | `src/stats/` modules |
| Modify visualizations | Individual plotting functions |
| Export to different format | `export_results_to_latex()` function |

### Citation

If you use this code in your research, please cite:

```bibtex
@article{foundation_plr_2026,
  title={Foundation Models for Pupillary Light Reflex Analysis in Glaucoma Screening},
  author={...},
  journal={...},
  year={2026}
}
```